In [1]:
from neo4j import GraphDatabase
from sklearn.metrics.pairwise import pairwise_distances,cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# from sklearn.metrics import pairwise_distances

Connect to Neo4j database


In [2]:
host = 'bolt://localhost:7687'
username = 'neo4j'
password = 'Jaehyun14*'

conn = GraphDatabase.driver(host,auth=(username,password))


Input the title

In [68]:
inputText = input("Enter title: ")

*Processing......*

In [69]:
with conn.session() as session:
    res = "MATCH (a:Anime) RETURN a.title AS title, a.genre AS genre, a.rating as rating,a.studio AS studio, a.show_type AS show_type"

    query = session.run(res)

    data = [(row['title'],row['genre'],row['rating'],row['studio'],row['show_type'])for row in query]

    empList = list()
    for title,genre,rating,studio,show_type in data:
        genre_split = [i for i in genre]
        empList.append((title,", ".join(genre_split),rating,studio,show_type))

    genres = [genre for _, genre, _, _ ,_ in empList]

    count_vec = CountVectorizer(binary=True)
    matrix = count_vec.fit_transform(genres).todense()
    np_array = np.asarray(matrix)

    similarity_matrix =  cosine_similarity(np_array)
    title_index = None
    for i, (title, _, _,_,_) in enumerate(empList):
        if title == inputText :
            title_index = i
            break

    similarity_scores = similarity_matrix[title_index]
    print(similarity_matrix[title_index])
    
    top_indices = similarity_scores.argsort()[::1][:-7:-1]
    recList = []
    
    recommendation = [(empList[i], similarity_scores[i]) for i in top_indices]
    recList.append(recommendation)

    recommendations = recList[-6:]




[0.         0.         0.         0.18898224 0.53452248 0.11952286
 0.26726124 0.37796447 0.65465367 0.         0.75592895 1.
 0.         0.37796447 0.18898224 0.         0.         0.21821789
 0.26726124 0.37796447 0.21821789 0.21821789 0.65465367 0.
 0.53452248 0.37796447 0.53452248 0.37796447 0.65465367 0.        ]


Results

In [70]:
for i in recommendations:
    del i[0]
    for items in i:
        items = list(items)
        show = items
        print(show)

        
    

[('Lucifer and the Biscuit Hammer', 'Action, Adventure, Comedy, Drama', 5.64, 'NAZ', 'TV Show'), 0.7559289460184544]
[('Shanhai Jing: Zaijian Gaoshou', 'Adventure, Comedy, Fantasy', 'N/A', 'N/A', 'Movie'), 0.6546536707079772]
[('New Gods: Yang Jian', 'Action, Adventure, Fantasy', 6.97, 'Light Chaser Animation Studios', 'Movie'), 0.6546536707079772]
[('New Gods: Yang Jian', 'Action, Adventure, Fantasy', 6.97, 'Light Chaser Animation Studios', 'Movie'), 0.6546536707079772]
[('My Precious Lunch', 'Action, Fantasy', 6.35, 'N/A', 'Movie'), 0.5345224838248487]
